In [1]:
import torch
import torchvision
import models
from PIL import Image
import numpy as np

In [2]:
def pil_loader(path):
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

In [3]:
# im_path = './data/D_01.jpg'
# im_path = './data/D_02.jpg'
# im_path = './data/D_03.jpg'
# im_path = './data/1000.jpg'
# im_path = './data/1001.jpg'
im_path = './data/1002.JPG'
model_hyper = models.HyperNet(16, 112, 224, 112, 56, 28, 14, 7).cuda()
model_hyper.train(False)
# load our pre-trained model on the koniq-10k dataset
# model_hyper.load_state_dict((torch.load('./pretrained/koniq_pretrained.pkl')))
model_hyper.load_state_dict((torch.load('./models/'+'livec_1685879565'+'.pth')), strict=False)

transforms = torchvision.transforms.Compose([
                    torchvision.transforms.Resize((512, 384)),
                    torchvision.transforms.RandomCrop(size=224),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                                     std=(0.229, 0.224, 0.225))])

# random crop 10 patches and calculate mean quality score
pred_scores = []
for i in range(10):
    img = pil_loader(im_path)
    img = transforms(img)
    img = torch.tensor(img.cuda()).unsqueeze(0)
    paras = model_hyper(img)  # 'paras' contains the network weights conveyed to target network

    # Building target network
    model_target = models.TargetNet(paras).cuda()
    for param in model_target.parameters():
        param.requires_grad = False

    # Quality prediction
    pred = model_target(paras['target_in_vec'])  # 'paras['target_in_vec']' is the input to target net
    pred_scores.append(float(pred.item()))
score = np.mean(pred_scores)
# quality score ranges from 0-100, a higher score indicates a better quality
print('Predicted quality score: %.2f' % score)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/ec2-user/.cache/torch/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

Predicted quality score: 0.05


/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
